# Before you begin

For this code to work, you need to have set the option to add Minio credentials when you started the server

![Minio Credentials Option](kubeflow-minio-option.png)

# Upload and download files from Bucket Storage!

Many of these examples are straight from the Minio Python SDK.

The difference is that instead of creating your own MinioClient object,
you can just and run

```python
import daaas_storage
minio_client = daaas_storage.get_minimal_client()
```

Then you proceed as in the examples!

## Get connected

In [ ]:
import daaas_storage
import os
#minio_client = daaas_storage.get_minimal_client()
minio_client = daaas_storage.get_premium_client()
#minio_client = daaas_storage.get_pachyderm_client()

## Create your bucket

In [ ]:
# Your bucket goes here.
BUCKET = 'blair-drummond'

In [ ]:
if not minio_client.bucket_exists(BUCKET):
    # ignore the location. It's a dummy value. But you do need it.
    minio_client.make_bucket(BUCKET, location="us-west-1")
else:
    print("Your bucket already exists.")

## Upload a file

In [ ]:
from progress import Progress
from minio.error import ResponseError

FILE='example.txt'

# Notice that it is "in a folder", but we didn't make a folder
# That's because object storage doesn't have folders. You, just
# put a slash in the name to "see a folder" in the GUI
NEW_FILE_NAME='examples/Happy-DAaaS-Bird.txt'

# Upload a file with progress.
progress = Progress()
try:
    with open(FILE, 'rb') as file_data:
        file_stat = os.stat(FILE)
        minio_client.put_object(
            BUCKET, 
            NEW_FILE_NAME,
            file_data,
            file_stat.st_size, 
            progress=progress
        )
except ResponseError as err:
    print(err)

## List Objects

In [ ]:
# List all object paths in bucket that begin with my-prefixname.
# Prefixes are basically how you select folders.
objects = minio_client.list_objects(
    BUCKET, 
    prefix='examples/',
    recursive=True
)

for obj in objects:
    print(obj.bucket_name, obj.object_name.encode('utf-8'), obj.last_modified,
          obj.etag, obj.size, obj.content_type)

## Download a file

In [ ]:
# Get a full object
try:
    data = minio_client.get_object(BUCKET, NEW_FILE_NAME)
    with open("downloaded_file.txt", 'wb') as file_data:
        for d in data.stream(32*1024):
            file_data.write(d)
except ResponseError as err:
    print(err)

# That's it!

You've seen how to upload, list, and download files.
You can do more things! For more advanced usage, check out

https://github.com/minio/minio-py

And don't forget that you can also do this all on the commandline with `mc`